<h3>What is Optical Flow?<h3>

Optical flow is a task of per-pixel motion estimation between two consecutive frames in one video. Basically, the Optical Flow task implies the calculation of the shift vector for pixel as an object displacement difference between two neighboring images. The main idea of Optical Flow is to estimate the object’s displacement vector caused by it’s motion or camera movements.

![Optical flow](Opticalflow.JPG)

<h3>Theoretical basics<h3>

Let’s assume that we have a gray-scale image – the matrix with pixel intensity. We define the function I(x,y,t), where x,y – pixel coordinates and t – frame number. The I(x,y,t) function defines the exact pixel intensity at the frame t.

To start with, we assume that the object displacement doesn’t change the pixels intensity that belongs to the exact object, it means that $I(x,y,t) = I(x+ \Delta x, y+ \Delta y, t + \Delta t)$. In our case $\Delta t = 1$. The major concern is to find the motion vector $(\Delta x, \Delta y)$. Let’s take a look at graphical representation:

![move](move.JPG)

Using Taylor series expansion we can rewrite $I(x,y,t) - I_(x+ \Delta x, y+ \Delta y, t + \Delta t) = 0$ as $I'_x u + I'_y v = - I'_t$, where $u = \frac{dx}{dt}, v = \frac{dy}{dt}$, and $I'_x, I'_y$ are image gradients. It is important that here we assume that parts of higher-order Taylor series are negligible, so this is a function approximation using only first-order Taylor’s expansion. The pixel motion difference between two frames I_1 and I_2 can be written as $I_1 - I_2 \approx I'_x u + I'_y v + I'_t$. Now, we have two variables u, v and only one equation, so we can’t solve the equation right now, but we can use some tricks which will be disclosed in the following algorithms.

<h3>Optical Flow applications<h3>

Optical Flow can be used in many areas where the object’s motion information is crucial. Optical Flow is commonly found in video editors for compression, stabilization, slow-motion, etc. Also, Optical Flow finds its application in Action Recognition tasks and real-time tracking systems.

<h3>Lucas-Kanade method<h3>